In [3]:
import torch
import pandas as pd
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import datasets
from datasets import load_dataset, load_from_disk
import logging
import datetime
log_filename = datetime.datetime.now().strftime("log_%Y-%m-%d_%H-%M-%S.txt")
# level=logging.DEBUG
# logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(format='%(asctime)s - %(filename)s[line:%(lineno)d] - %(levelname)s: %(message)s',
                    level=logging.DEBUG,
                    filename=log_filename,
                    filemode='w')


MODEL_PATH="Model/"
CACHE_DIR= "datasets/"
LOG_DIR = "LOGS/"
# MODEL_NAME = "models--deepset--roberta-base-squad2"
# MODEL_POS = MODEL_PATH+MODEL_NAME
# # MODEL_POS = "Model/models--deepset--roberta-base-squad2/snapshots/e84d19c1ab20d7a5c15407f6954cef5c25d7a261/"
# print(MODEL_POS)
#加载模型和分词器

In [4]:

dataset = load_dataset("ms_marco",'v1.1',cache_dir = CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2",cache_dir=MODEL_PATH)
model = AutoModelForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

In [16]:
print(dataset.shape)

{'validation': (10047, 6), 'train': (82326, 6), 'test': (9650, 6)}


In [17]:
print(dataset["test"].num_rows)

9650


In [18]:
print(dataset["test"][1])

{'answers': ['Fossil fuels are basically the remains of animals and plants and these are good energy resources. The three main fossil fuels are natural gas, oil, and coal. Fossil fuels are low in cost and are very important resources for our economy. Fossil fuels are used to generate electricity used as fuels for transportation.'], 'passages': {'is_selected': [0, 1, 0, 0, 0, 0, 0, 0, 0], 'passage_text': ['The biggest advantage of using fossil fuels is that they can be easily stored and transported from one place to another. Large reserves of coal are therefore taken from the coal mines to the industries which are acres away from the mines. The petroleum is also taken to too far off power stations to produce energy. Fossil fuels are the highest producers of calorific value in terms of energy. This is also one of the reasons why they are still preferred over the renewable sources of energy or the alternative source', 'Benefits of fossil fuels. Fossil fuels are basically the remains of an

In [19]:
print(dataset.column_names)

{'validation': ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'], 'train': ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'], 'test': ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers']}


In [17]:
test_set = dataset["test"]
res = np.zeros((test_set.num_rows,test_set.num_columns + 1))
print(res.shape)
row = test_set[1]
print('test_set type :' + str(test_set))
print('row type: '+str(type(row)))
# corr_ans = row[0]
print('passages keys')
print(row['passages'].keys())
# print(len(row['passages']['is_selected']))
# print(row['passages']['is_selected'])
print(type(row['passages']['is_selected']))
print(type(test_set))

(9650, 7)
test_set type :Dataset({
    features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
    num_rows: 9650
})
row type: <class 'dict'>
passages keys
dict_keys(['is_selected', 'passage_text', 'url'])
<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>


In [1]:

for i in range(test_set.num_rows):
    row = test_set[i]
    # print(type(row))
    # print(type(row['query']))
    print(row['passages'].keys())
    inputs = tokenizer.encode_plus(row['query'], row['passages'], add_special_tokens=True, return_tensors="pt")
    output = model(**inputs)
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits) + 1
    gen_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    data['query_id'].append(row['query_id'])
    data['generated_answer'].append(gen_answer)
    data['correct_answer'].append(row['answer'])
    logging.debug(str(row['query_id'])+ ' answer ' + gen_answer)

NameError: name 'test_set' is not defined

In [31]:
data ={
    "query_id":[],
    "generated_answer":[],
    "correct_answer":[]
}
for row_item in test_set:
    selected_item = row_item['passages']['is_selected']
    passage_item = row_item['passages']['passage_text']
    # print('passage item type : ' + str(passage_item))

    passage_string = str()
    for idx in range(len(selected_item)):
        if selected_item[idx] == 1:
            passage_string += passage_item[idx]
    # print(input_string)
    inputs = tokenizer.encode_plus(row_item['query'], passage_string, add_special_tokens=True, return_tensors="pt")
    output = model(**inputs)
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits) + 1
    gen_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

    data['query_id'].append(row_item['query_id'])
    data['generated_answer'].append(gen_answer)
    data['correct_answer'].append(row_item['answers'])

    logging.debug('idx '+'query_id : ' + str(row_item['query_id']) + 'gen_answer :'+gen_answer)

df = pd.DataFrame(data)
df.to_csv("output.csv",index=True , encoding="utf-8")

KeyboardInterrupt: 

In [1]:

# 定义问题和上下文
question = "What's the capital of France?"
context = "France is a country in Europe. Its capital is Paris."
# 使用分词器处理文本
inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")

# 使用模型进行预测
output = model(**inputs)

# 获取答案的起始和结束位置
answer_start = torch.argmax(output.start_logits)
answer_end = torch.argmax(output.end_logits) + 1

# 将 token ID 转换回文本
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

print(answer)

2.1.2+cu118
True


In [2]:
from transformers import pipeline